# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

a         b     g           c           d            e  \
5   704.683590  0.064210  1234  704.747800  704.619380  1409.495601   
6   762.752604 -0.107346  1234  762.645258  762.859950  1525.290515   
7    87.235887 -0.235631  1234   87.000255   87.471518   174.000511   
8   121.956837 -0.089674  1234  121.867164  122.046511   243.734327   
9   331.312849  0.805248  1234  332.118097  330.507601   664.236194   
10  136.954361  0.486290  1234  137.440651  136.468071   274.881302   
11  664.246525 -0.691689  1234  663.554836  664.938214  1327.109673   
12  917.444685  0.401152  1234  917.845837  917.043534  1835.691674   
13  443.320131 -0.998264  1234  442.321867  444.318394   884.643734   
14   88.867430  0.231439  1234   89.098869   88.635990   178.197737   

              f  
5   1409.238760  
6   1525.719900  
7    174.943036  
8    244.093021  
9    661.015203  
10   272.936143  
11  1329.876427  
12  1834.087068  
13   888.636788  
14   177.271981

In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

objective_c  objective_d
5   -704.747800   704.619380
6   -762.645258   762.859950
7    -87.000255    87.471518
8   -121.867164   122.046511
9   -332.118097   330.507601
10  -137.440651   136.468071
11  -663.554836   664.938214
12  -917.845837   917.043534
13  -442.321867   444.318394
14   -89.098869    88.635990

In [13]:
# This is also available as a method
vocs.objective_data(data)

objective_c  objective_d
5   -704.747800   704.619380
6   -762.645258   762.859950
7    -87.000255    87.471518
8   -121.867164   122.046511
9   -332.118097   330.507601
10  -137.440651   136.468071
11  -663.554836   664.938214
12  -917.845837   917.043534
13  -442.321867   444.318394
14   -89.098869    88.635990

In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-704.74780025,  704.61937989],
       [-762.6452575 ,  762.85994981],
       [ -87.00025547,   87.47151798],
       [-121.86716356,  122.04651068],
       [-332.11809708,  330.50760131],
       [-137.44065091,  136.46807139],
       [-663.55483632,  664.93821356],
       [-917.84583694,  917.04353393],
       [-442.32186692,  444.3183942 ],
       [ -89.09886863,   88.63599044]])

In [15]:
vocs.constraint_data(data)

constraint_e  constraint_f
5    1407.495601  -1409.238760
6    1523.290515  -1525.719900
7     172.000511   -174.943036
8     241.734327   -244.093021
9     662.236194   -661.015203
10    272.881302   -272.936143
11   1325.109673  -1329.876427
12   1833.691674  -1834.087068
13    882.643734   -888.636788
14    176.197737   -177.271981

In [16]:
vocs.feasibility_data(data)

feasible_e  feasible_f  feasible
5        False        True     False
6        False        True     False
7        False        True     False
8        False        True     False
9        False        True     False
10       False        True     False
11       False        True     False
12       False        True     False
13       False        True     False
14       False        True     False

# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

A
1  6.000000e+66
2  6.000000e+66
3  6.000000e+66

In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

objective_c  objective_d
9         -1.0          inf
3         -2.0          inf
4         -3.0          inf
5         -4.0          inf